# QC samples

In [1]:
import sys
sys.path.append('../../')
import pandas as pd
import glob
from pathlib import Path
from config import CLEAN_DATA_DIR, OUTPUT_DIR, RAW_DATA_DIR
import db as db


from scripts.normalize_data import (
    csv_cleanup,
    update_metadata,
    get_taxonomy_columns,
)

from scripts.shared_utils import (
    log_df
)

In [2]:
CLEAN_DATA_DIR

PosixPath('../../output/cleaned_data')

In [19]:
clean_data_paths = [
    CLEAN_DATA_DIR /'LIMS/Micropal_CSV_1', 
    CLEAN_DATA_DIR /'LIMS/Micropal_CSV_2', 
    CLEAN_DATA_DIR /'LIMS/Micropal_CSV_3', 
    CLEAN_DATA_DIR /'LIMS/Micropal_CSV_4', 
    CLEAN_DATA_DIR /'LIMS/Micropal_CSV_revised', 
]

lith_data_path = CLEAN_DATA_DIR /'LIMS'/'Lithology_CSV'



date = '2022-08-08'


taxa_list_file = OUTPUT_DIR/'taxa'/'LIMS'/f"taxa_list_{date}.csv"
taxa_crosswalk_list_file = OUTPUT_DIR/'taxa'/'LIMS'/f"taxa_crosswalk_{date}.csv"

metadata_file = OUTPUT_DIR/'metadata'/'LIMS'/'Micropal_changes.csv' 



## check samples with no taxa

In [86]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_groups,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,add_expedition_section_cols,update_sample_col,update_top_bottom,add_missing_cols,clean_up_taxa_values,clean_up_taxa_metadata_values,split_dextral_sinistral
0,363-U1482A-Benthic_Forams.csv,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False,False,False,True,False
1,320_U1336A_Nannofossils_2.csv,LIMS/Micropal_CSV_1/320_U1336A_Nannofossils_2.csv,nannofossils,False,False,False,True,False,False,False,True,False,False,False,False,False
2,375_U1518F_planktic_forams.csv,LIMS/Micropal_CSV_1/375_U1518F_planktic_forams...,planktic_forams,False,False,False,True,False,False,True,False,False,False,False,False,False
3,320_U1334A_Nannofossils_1.csv,LIMS/Micropal_CSV_1/320_U1334A_Nannofossils_1.csv,nannofossils,False,False,False,True,False,False,False,True,False,False,False,False,False
4,318_U1358B_Palynology.csv,LIMS/Micropal_CSV_1/318_U1358B_Palynology.csv,palynology,False,False,False,False,False,False,False,True,False,False,False,False,False


In [88]:
taxa_df = pd.read_csv(taxa_crosswalk_list_file, dtype=str)
log_df(taxa_df, 2)
# 5380

(5380, 18)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,name comment field,Comment,Notes (change to Internal only notes?),comments,eodp_id
0,NaN,NaN,Euuvigerina,NaN,NaN,NaN,miozea,NaN,NaN,NaN,Euuvigerina miozea,benthic_forams,Euuvigerina miozea (group) >100 m,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",>100 m group,0
1,NaN,NaN,Euuvigerina,NaN,NaN,NaN,rodleyi,NaN,NaN,NaN,Euuvigerina rodleyi,benthic_forams,Euuvigerina rodleyi (group) >50 m,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",>50 m group,1


In [89]:
taxa = list(taxa_df['verbatim_name'])
len(taxa)

5380

In [90]:
file = CLEAN_DATA_DIR /'LIMS/Micropal_CSV_2/368_U1501D_nannofossils.csv'


def get_no_taxa_samples(file):
    df = pd.read_csv(file, dtype=str)
    df.dropna(how='all', axis=1, inplace=True)

    file_taxa = set(df.columns).intersection(taxa)

    tmp_df = df[list(file_taxa)]
    notaxa_df = df.loc[tmp_df[tmp_df.isna().all(1)].index]
    
    return list(notaxa_df['eodp_id'])



In [91]:

eodp_ids = []


for index, row in metadata.iterrows():
#     if index > 10:
#         continue
        
    file = CLEAN_DATA_DIR/row['path']
    
    res =  get_no_taxa_samples(file)

    eodp_ids = eodp_ids + res
    
len(eodp_ids)

11421

In [92]:
sql = """
select  samples.eodp_id
from samples
left join samples_taxa on samples_taxa.sample_id = samples.id
where samples_taxa.id is null
and samples.data_source_type = 'micropal csv';
"""


eodp_ids_db = []
rows = db.fetch_all(sql)
for row in rows:
    eodp_ids_db.append(row['eodp_id'])
    
len(eodp_ids_db)

11421

In [93]:
set(eodp_ids) - set(eodp_ids_db)

set()

## create csv for samples with no taxa

samples no taxa, with abundance

In [119]:

sql = """
select samples.name as sample, samples_fields.sample_id as sample_id, 
original_name as original_field_name, fields.name as field_name, value,
samples.data_source_notes
from samples_fields 
join samples on samples.id = samples_fields.sample_id
join fields on samples_fields.field_id = fields.id 

where sample_id in (
    select  samples.id
    from samples
    left join samples_taxa on samples_taxa.sample_id = samples.id
    where samples_taxa.id is null
    and samples.data_source_type = 'micropal csv'
)
and fields.name ILIKE '%abundance%'
order by samples.data_source_notes, samples.id
;
"""

rows = db.fetch_all_dict(sql)

df = pd.DataFrame(rows)
log_df(df)

(10550, 6)


,sample,sample_id,original_field_name,field_name,value,data_source_notes
0,318-U1355A-1R-1-PAL-DIAT4,283078,Silicoflagellates group abundance,Silicoflagellate abundance,X,LIMS/Micropal_CSV_1/318_U1355A_Silicoflagellat...
1,318-U1355A-2R-1-W 77/77-DIAT2,283082,Silicoflagellates group abundance,Silicoflagellate abundance,X,LIMS/Micropal_CSV_1/318_U1355A_Silicoflagellat...
2,318-U1356A-39R-1-W 85/85-DIAT1,287314,Diatoms group abundance,Diatom abundance,B,LIMS/Micropal_CSV_1/318_U1356A_Diatoms_1.csv
3,318-U1356A-40R-1-W 57/58-DIAT3,287316,Diatoms group abundance,Diatom abundance,B,LIMS/Micropal_CSV_1/318_U1356A_Diatoms_1.csv
4,318-U1356A-42R-CC-PAL-DIAT4,287318,Diatoms group abundance,Diatom abundance,B,LIMS/Micropal_CSV_1/318_U1356A_Diatoms_1.csv


In [120]:
df.to_csv(OUTPUT_DIR/'tmp'/'group_abundance_for_samples_no_taxa.csv', index=False)

In [121]:

sql = """

select value, count(*)
from samples_fields 
join samples on samples.id = samples_fields.sample_id
join fields on samples_fields.field_id = fields.id 

where sample_id in (
    select  samples.id
    from samples
    left join samples_taxa on samples_taxa.sample_id = samples.id
    where samples_taxa.id is null
    and samples.data_source_type = 'micropal csv'
)
and fields.name ILIKE '%abundance%'
group by value
order by count desc
;
"""

rows = db.fetch_all_dict(sql)

df = pd.DataFrame(rows)
log_df(df)

(25, 2)


,value,count
0,B,8083
1,A,547
2,R,462
3,P,357
4,C,332


In [122]:
df.to_csv(OUTPUT_DIR/'tmp'/'group_abundance_for_samples_no_taxa_count.csv', index=False)

samples no taxa, no abundance, with other fields

In [123]:

sql = """

select samples.name as sample, samples_fields.sample_id as sample_id, 
original_name as original_field_name, fields.name as field_name, value,
samples.data_source_notes
from samples_fields 
join samples on samples.id = samples_fields.sample_id
join fields on samples_fields.field_id = fields.id 

where sample_id in (
select  samples.id
from samples
left join samples_taxa on samples_taxa.sample_id = samples.id
where samples_taxa.id is null
and samples.data_source_type = 'micropal csv'

except

select  samples.id
from samples_fields 
join samples on samples.id = samples_fields.sample_id
join fields on samples_fields.field_id = fields.id 
where fields.name ILIKE '%abundance%'
)
order by samples.data_source_notes, samples.id

"""

rows = db.fetch_all_dict(sql)

df = pd.DataFrame(rows)
log_df(df)

(1072, 6)


,sample,sample_id,original_field_name,field_name,value,data_source_notes
0,318-U1356A-49R-CC-PAL-DIAT4,287328,Diatom preservation fragmentation,Diatom preservation fragmentation,G,LIMS/Micropal_CSV_1/318_U1356A_Diatoms_1.csv
1,318-U1356A-49R-CC-PAL-DIAT4,287328,Diatom preservation dissolution,Diatom preservation dissolution,P,LIMS/Micropal_CSV_1/318_U1356A_Diatoms_1.csv
2,318-U1356A-55R-CC-PAL-DIAT4,287335,Diatom preservation fragmentation,Diatom preservation fragmentation,P,LIMS/Micropal_CSV_1/318_U1356A_Diatoms_1.csv
3,318-U1356A-55R-CC-PAL-DIAT4,287335,Diatom preservation dissolution,Diatom preservation dissolution,P,LIMS/Micropal_CSV_1/318_U1356A_Diatoms_1.csv
4,318-U1356A-57R-CC-PAL-DIAT7,287337,Diatom preservation dissolution,Diatom preservation dissolution,P,LIMS/Micropal_CSV_1/318_U1356A_Diatoms_1.csv


In [124]:
df.to_csv(OUTPUT_DIR/'tmp'/'samples_no_taxa_no_abundance.csv', index=False)

samples no taxa, no abundance, no other fields

In [127]:
sql = """
select samples.name as sample, id as sample_id, 
samples.data_source_notes
from samples
 
where id in (
select  samples.id
from samples
left join samples_taxa on samples_taxa.sample_id = samples.id
where samples_taxa.id is null
and samples.data_source_type = 'micropal csv'

intersect

select  samples.id
from samples
left join samples_fields on samples_fields.sample_id = samples.id
where samples_fields.id is null
and samples.data_source_type = 'micropal csv'
)

order by samples.data_source_notes, samples.id
;

"""


rows = db.fetch_all_dict(sql)

df = pd.DataFrame(rows)
log_df(df)

(163, 3)


,sample,sample_id,data_source_notes
0,318-U1356A-46R-CC-PAL-FORAM,283085,LIMS/Micropal_CSV_1/318_U1356A_Benthic_Forams.csv
1,318-U1356A-49R-CC-PAL,283086,LIMS/Micropal_CSV_1/318_U1356A_Benthic_Forams.csv
2,318-U1356A-50R-CC-PAL-FORAM,283087,LIMS/Micropal_CSV_1/318_U1356A_Benthic_Forams.csv
3,318-U1356A-53R-CC-PAL-FORAM,283088,LIMS/Micropal_CSV_1/318_U1356A_Benthic_Forams.csv
4,318-U1356A-73R-CC-PAL-FORAM,283089,LIMS/Micropal_CSV_1/318_U1356A_Benthic_Forams.csv


In [128]:
df.to_csv(OUTPUT_DIR/'tmp'/'samples_no_taxa_no_abundance_no_other_fields.csv', index=False)

## check taxa with no pbdb name

In [43]:
sql = """
SELECT pbdb_taxon_id, name
FROM taxa  WHERE ("pbdb_taxon_id" IS NOT NULL) AND ("pbdb_taxon_name" IS NULL)
"""

records = []
rows = db.fetch_all(sql)
for row in rows:
    records.append({'pbdb_taxon_id': row['pbdb_taxon_id'],
                    'name': row['name']})
    
len(rows)

14

In [41]:
file = RAW_DATA_DIR/'PI_processed_files'/'LIMS_Micropal_CSV_4_taxa_ADDTL_TAXA.csv'

df = pd.read_csv(file, usecols=['pbdb_taxon_id', 'verbatim_name'])
df.drop_duplicates(inplace=True)

log_df(df)


(20, 2)


,verbatim_name,pbdb_taxon_id
0,Lonchosphaera spicata,85178
1,Poulpus spp.-like,614
2,Lithostrobus cuspidatus,561
3,Gondwanaria dogieli,432661
4,Tetraplecta pinigera/Euscenium corynephorum,66455


In [44]:
for index, record in enumerate(records):
    new_df = df[
        (df['pbdb_taxon_id'] == record['pbdb_taxon_id'])
    ]
    print(index, new_df['verbatim_name'].values, record['name'])

0 ['Hantkenina australis'] Hantkenina australis
1 ['Grobolotalia spp.'] Globorotalia spp.
2 ['Lonchosphaera spicata'] Lonchosphaera spicata
3 ['Poulpus spp.-like'] Poulpus spp.
4 ['Lithostrobus cuspidatus'] Lithostrobus cuspidatus
5 ['Gondwanaria dogieli'] Gondwanaria dogieli
6 ['Tetraplecta pinigera/Euscenium corynephorum'] Plagiacanthidae indet.
7 ['Litharachnium tentorium'] Litharachnium tentorium
8 ['Streblacantha circumtexta'] Streblacantha circumtexta
9 ['Eucyrtidium teuscheri'] Eucyrtidium teuscheri
10 ['Verticillata hexacantha'] Verticillata hexacantha
11 ['Cypassis irregularis'] Cypassis irregularis
12 ['Prunopyle antarctica'] Prunopyle antarctica
13 ['Big Lithomelissa sp. A (Antarctissa like)'] Lithomelissa sp. A


## check if all files are imported

taxa files

In [41]:
files = []

for clean_data_path in clean_data_paths:
    raw_csvs = glob.glob(f"{clean_data_path}/*.csv")
    for csv in raw_csvs:
        files.append(csv.split('cleaned_data/')[1])

files[0:3]

['LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv',
 'LIMS/Micropal_CSV_1/320_U1336A_Nannofossils_2.csv',
 'LIMS/Micropal_CSV_1/375_U1518F_planktic_forams.csv']

In [42]:
len(files)

1253

In [43]:
sql = """
select distinct(data_source_notes) as file
from samples 
where data_source_type = 'micropal csv';
"""

db_files = []
rows = db.fetch_all(sql)
for row in rows:
    db_files.append(row['file'])
    
db_files[0:3]

['LIMS/Micropal_CSV_3/339_nannofossils_U1387C.csv',
 'LIMS/Micropal_CSV_4/317_U1353_planktic_forams.csv',
 'LIMS/Micropal_CSV_3/341_diatoms_U1417E.csv']

In [44]:
len(db_files)

1253

In [45]:
len(db_files) == len(files)

True

lithology files

In [33]:
files = []


raw_csvs = glob.glob(f"{lith_data_path}/*.csv")
for csv in raw_csvs:
    files.append(csv.split('/')[-1])

files[0:3]

['361_macroscopic_U1474D.csv',
 '323 Core Description Template_U1341A.csv',
 '361_macroscopic_U1479C.csv']

In [34]:
len(files)

518

In [35]:
sql = """
select distinct(data_source_notes) as file
from samples 
where data_source_type = 'lithology csv';
"""

db_files = []
rows = db.fetch_all(sql)
for row in rows:
    db_files.append(row['file'])
    
db_files[0:3]

['342_sediment_U1406A.csv',
 '359-U1466A_macroscopic.csv',
 '340_sediment_U1396C.csv']

In [36]:
len(db_files)

516

In [40]:
len(db_files) == len(files)

False

In [37]:
set(db_files) - set(files)

set()

In [39]:
set(files) - set(db_files)

{'329_sediment_U1369D.csv', '362_macro_U1480D.csv'}

## check if taxa column for all files are imported

get files with no samples_taxa

In [60]:
sql = """
select distinct(data_source_notes) from samples where data_source_type = 'micropal csv'
except
select distinct(data_source_notes) from samples_taxa;
"""

db_files = []
rows = db.fetch_all(sql)
for row in rows:
    db_files.append(row['data_source_notes'])
    
db_files[0:3]

['LIMS/Micropal_CSV_2/371_U1511B_benthic_forams.csv',
 'LIMS/Micropal_CSV_2/371_U1511A_benthic_forams.csv',
 'LIMS/Micropal_CSV_2/346_U1423C_nannofossils.csv']

In [61]:
len(db_files)

48

get all verbatim_name

In [62]:
sql = """
select distinct(verbatim_name)
from taxa_crosswalk;
"""
taxa = []
rows = db.fetch_all(sql)
for row in rows:
    taxa.append(row['verbatim_name'])
    
taxa[0:3]

['Valkyria pukapuka', 'Spirotextularia fistulosa', 'Monalysidium spp.']

print file name if there are taxa columns with abundance values

In [63]:
errors = False 

for file in db_files:
    path = CLEAN_DATA_DIR/file
    df = pd.read_csv(path)
    df.dropna(axis=1, how='all', inplace=True)
    
    if len(set(df.columns).intersection(set(taxa))) > 0:
        print(path)
        errors = True 
        
if not errors:
    print('all files ok')

all files ok


## Look for duplicate sample names in all mircopal files

In [10]:
data =[]
target_columns = [
    'Sample', 
    'Top [cm]', 
    'Bottom [cm]', 
    'Top Depth [m]', 
    'Bottom Depth [m]'
]
for clean_data_path in clean_data_paths:
    raw_csvs = glob.glob(f"{clean_data_path}/*.csv")

    for path in raw_csvs:
        content = pd.read_csv(path)
        
        new_df = content[content.duplicated(subset=['Sample'])]
        for index, row in new_df.iterrows():
            data.append({'sample': row['Sample'],  'path': path})

In [11]:
new_df = pd.DataFrame(data)
new_df.shape

(636, 2)

In [12]:
new_df

,sample,path
0,363-U1487A-4H-CC-PAL-NANNO,../../output/cleaned_data/LIMS/Micropal_CSV_1/...
1,363-U1487A-5H-CC-PAL-NANNO,../../output/cleaned_data/LIMS/Micropal_CSV_1/...
2,363-U1483A-1H-1-A 0/0-NANNO,../../output/cleaned_data/LIMS/Micropal_CSV_1/...
3,363-U1483A-9H-6-W 50/50-NANNO,../../output/cleaned_data/LIMS/Micropal_CSV_1/...
4,363-U1483A-11H-2-W 50/50-NANNO,../../output/cleaned_data/LIMS/Micropal_CSV_1/...
...,...,...
631,323-U1345A-1H-1-nan,../../output/cleaned_data/LIMS/Micropal_CSV_4/...
632,323-U1345A-1H-2-nan,../../output/cleaned_data/LIMS/Micropal_CSV_4/...
633,323-U1345A-1H-2-nan,../../output/cleaned_data/LIMS/Micropal_CSV_4/...
634,323-U1345A-2H-1-nan,../../output/cleaned_data/LIMS/Micropal_CSV_4/...


In [14]:
# new_df.to_csv('../../output/tmp/dup_sample_names.csv')

## import all samples into db

In [37]:
import math

In [39]:
for clean_data_path in clean_data_paths:
    raw_csvs = glob.glob(f"{clean_data_path}/*.csv")

    for path in raw_csvs:
        filename = path.split('/')[2]
        content = pd.read_csv(path)

        for index, row in content.iterrows():
             if type(row['Sample']) is str and (type(row['Top [cm]']) is int or type(row['Top [cm]']) is float):

                 top =  0 if math.isnan(row['Top [cm]']) else row['Top [cm]']
                 sample = row['Sample'].strip()
                 sql = f"INSERT INTO staging.samples (name,top,bottom,top_depth,bottom_depth, created_at, data_source_notes)  VALUES (\'{sample}\', {top} , {row['Bottom [cm]']} , {row['Top Depth [m]']} ,{row['Bottom Depth [m]']}, now(), \'{filename}\');"
                 # db.execute(sql);
             else:
                print(row['Sample'], row['Top [cm]'], row['Bottom [cm]'], row['Top Depth [m]'], row['Bottom Depth [m]'], path )


print('done')


## check if expeditions matches samples and file name

In [4]:
# get expedition name from all the joins
sql1 = """
select distinct expeditions.name, samples.data_source_type
from samples 
left join sections on sections.id =  samples.section_id
left join cores on cores.id = sections.core_id
left join holes on holes.id = cores.hole_id
left join sites on sites.id = holes.site_id
left join expeditions on expeditions.id = sites.expedition_id
"""


# get expedition name from samples.raw_data
sql2 = """
select distinct samples.raw_data ->> 'Exp' as name, samples.data_source_type
from samples 
"""

# get expedition name from samples.name
sql3 = """
select distinct(LEFT(name, 3)) as name, samples.data_source_type
from samples
"""

sql = sql1 + ' except ' + sql2 
rows = db.fetch_all_dict(sql)
print(rows)

sql = sql2 + ' except ' + sql1 
rows = db.fetch_all_dict(sql)
print(rows)


sql = sql2 + ' except ' + sql3
rows = db.fetch_all_dict(sql)
print(rows)

sql = sql3 + ' except ' + sql2 
rows = db.fetch_all_dict(sql)
print(sql, rows)

[]
[]
[]

select distinct(LEFT(name, 3)) as name, samples.data_source_type
from samples
 except 
select distinct samples.raw_data ->> 'Exp' as name, samples.data_source_type
from samples 
 [RealDictRow([('name', None), ('data_source_type', 'lithology csv')]), RealDictRow([('name', None), ('data_source_type', 'micropal csv')])]


## check if sample name matches individual parts

In [7]:
sql="""select name, tmp_name, data_source_notes 
from (
select name, id, data_source_notes,
CASE
    WHEN raw_data ->> 'Section' != '' THEN 
    (raw_data ->> 'Exp') || '-' 
    || (raw_data ->> 'Site') 
    || (raw_data ->> 'Hole') || '-' 
    || (raw_data ->> 'Core') 
    || (raw_data ->> 'Type') || '-' 
    || (raw_data ->> 'Section')
    ELSE
    (raw_data ->> 'Exp') || '-' 
    || (raw_data ->> 'Site') 
    || (raw_data ->> 'Hole') || '-' 
    || (raw_data ->> 'Core') 
    || (raw_data ->> 'Type') 
END tmp_name
from samples
) as foo
where  name not LIKE tmp_name || '%';
"""
rows = db.fetch_all_dict(sql)

df = pd.DataFrame(rows)

In [8]:
df

,name,tmp_name,data_source_notes
0,350-U1437E-9R-CC-PAL-FORAM,350-U1437E-9H-CC,LIMS/Micropal_CSV_2/350_U1437E_planktic_forams...
1,350-U1437E-10R-CC-PAL-FORAM,350-U1437E-10H-CC,LIMS/Micropal_CSV_2/350_U1437E_planktic_forams...
2,350-U1437E-11R-CC-PAL-FORAM,350-U1437E-11H-CC,LIMS/Micropal_CSV_2/350_U1437E_planktic_forams...
3,350-U1437E-13R-1-PAL-FORAM,350-U1437E-13H-1,LIMS/Micropal_CSV_2/350_U1437E_planktic_forams...
4,350-U1437E-14R-CC-PAL-FORAM,350-U1437E-14H-CC,LIMS/Micropal_CSV_2/350_U1437E_planktic_forams...
...,...,...,...
2907,350-U1436B-14H-2-A,350-U1436B-2H-3,350_macroscopic_U1436B.csv
2908,318-U1356A-96R-CC-PAL,318-U1356A-96RR-CC,LIMS/Micropal_CSV_4/318_U1356A_nannofossils.csv
2909,341-U1417A,341-U1417A-,LIMS/Micropal_CSV_3/341_radiolarians_U1417A.csv
2910,341-U1417A-20H-CC-PAL,341-U1418A-20H-CC,LIMS/Micropal_CSV_3/341_radiolarians_U1417A.csv
